In [39]:
!pip install fuzzywuzzy

In [40]:
import pandas as pd  
import re
import csv 
import datetime
import string 
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
import nltk as nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harisnaveed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
people = 'Donald Trump', 'Joe Biden','Kamala Harris', 'Mike Pence'
republican_president = 'Donald Trump'
republican_vice_president = "Pence$"
democratic_president = "Biden$"
democratic_vice_president = "Harris$"
host1 = "Welker$"
host2 = "Stephanopoulos$"
host3 = "Wallace$"
host4 = "Page$"
host5 = "Guthrie$"
race = ["race", "equality", "racial", "racist", "african-american", "mr. floyd", "radical"]
violence = ["radical", "mr. floyd", "racist", "guns", "cops", "protest"]
economy = ["inflation", "economy", "demand", "capital", "credit", "economics", "booming", "market", 
           "interest", "deflation", "poverty", "jobs", "job", "unemployment", "recession", "incentives",
           "infrastructure", "paying", "reserve"]
health_care = ["health care", "medical", "obamacare", "insurance", "affordable", "care", "act", 'mecaid']
covid = ["covid-19", "covid", "virus", "Xi", "china", "cases"]
supreme_court = ['supreme court', "judge", "case"]
intro = ["hello", "good","well", "how are you", "evening", "beginning", "how are you doing", "end"]

In [42]:
#file_dataframe = pd.read_csv('archive/us_election_2020_biden_town_hall.csv')
file_dataframe = pd.read_csv('archive/us_election_2020_1st_presidential_debate.csv')

In [43]:
text = file_dataframe['text']
minute_series = file_dataframe['minute']
speaker = file_dataframe['speaker']

In [44]:
def word_count_func(text):
    words = text.split()
    return len(words)

In [45]:
# Getting word count for each row
word_count = text.apply(word_count_func)

In [46]:
def changing_time_to_integer(minute_one):
    minute_one = str(minute_one)
    print(minute_one)
    # if minute_one.format("f{}:{}"):
    #     time = datetime.datetime.strptime(minute_one, '%M:%S')
    
    # date_time = time - datetime.datetime(1900, 1, 1)
    # return int(date_time.total_seconds())
    # if len(minute_one.split(":")) == 2:
    #     time = datetime.datetime.strptime(minute_one, '%M:%S')
    # else:
    #     time = datetime.datetime.strptime(minute_one, '%H:%M:%S')

    # date_time = time - datetime.datetime(1900, 1, 1)
    
    # return int(date_time.total_seconds())


In [47]:
time_in_seconds = minute_series.apply(changing_time_to_integer)
time_in_seconds

01:20
02:10
02:49
02:51
02:51
03:11
04:01
04:53
05:22
05:29
05:34
05:36
06:12
07:08
07:34
08:01
08:02
08:24
08:25
08:29
08:31
08:32
08:33
08:58
09:00
09:02
09:05
09:08
09:09
09:11
09:12
09:39
09:47
09:48
09:58
10:00
10:04
10:04
10:05
10:06
10:11
10:25
10:31
10:32
10:33
10:34
10:35
10:36
10:40
10:51
10:51
11:05
11:05
11:07
11:08
11:11
11:11
11:25
11:26
11:28
11:31
11:32
11:35
11:36
11:38
11:38
11:44
11:44
11:46
11:52
11:53
12:16
12:40
12:40
12:41
12:42
13:03
13:04
13:06
13:17
13:17
13:23
13:25
13:26
13:29
13:32
13:57
14:04
14:05
14:06
14:10
14:10
14:12
14:12
14:14
14:17
14:18
14:21
14:23
14:23
14:25
14:25
14:34
14:41
14:43
14:44
14:46
14:47
14:51
14:55
14:57
14:58
14:58
14:59
15:01
15:13
15:14
15:20
15:21
15:41
15:45
15:46
15:47
15:49
15:54
15:59
16:42
16:45
16:45
16:47
16:48
16:50
16:52
16:52
17:13
17:15
17:16
17:55
18:05
18:07
18:08
18:09
18:12
18:17
18:18
18:18
18:20
18:23
18:25
18:25
18:27
18:30
18:35
18:37
18:37
18:40
19:33
20:30
20:43
20:43
20:57
20:57
20:57
20:57
21:30
21:31
21:3

0      None
1      None
2      None
3      None
4      None
       ... 
784    None
785    None
786    None
787    None
788    None
Name: minute, Length: 789, dtype: object

In [48]:
initial = -1
time_in_seconds_list = list()
for time in list(time_in_seconds):
    if time < initial:
        time = time + initial    
    initial = time
    time_in_seconds_list.append(time)
time_in_seconds = pd.Series(time for time in time_in_seconds_list)


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [76]:
def checking_speaker_party(speaker):
    speaker = speaker.strip()
    
    if re.search(republican_president, speaker) != None:
        return "Republican"
    elif re.search(republican_vice_president, speaker) != None:
        return "Republican"
    elif re.search(democratic_president, speaker) != None or re.search(democratic_vice_president, speaker) != None:
        return "Democratic" 
    elif re.search(host1, speaker) != None or re.search(host2, speaker) != None \
        or re.search(host3, speaker) != None or re.search(host4, speaker) != None \
        or re.search(host5, speaker):
        return "Host"
    else:
        return "Citizen"

In [77]:
# Getting party types in the debates
party_type = speaker.apply(checking_speaker_party)

In [52]:
def checking_number_of_questions(text):
    count = 0
    for char in text:
        if char == "?":
            count += 1
    return count

In [53]:
number_of_questions = text.apply(checking_number_of_questions)

In [78]:
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [79]:
stop = set(stopwords.words("english"))
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [80]:
def summary_columns(text, list_of_summary_words):       
      res = any(word in text for word in list_of_summary_words)       
      return res

In [81]:
economy_true_or_false = text.apply(summary_columns, args=(economy,))
health_care_true_or_false = text.apply(summary_columns, args=(health_care,))
covid_true_or_false = text.apply(summary_columns, args=(covid,))
supreme_court_true_or_false = text.apply(summary_columns, args=(supreme_court,))
# intro_ending_true_or_false = text.apply(summary_columns, args=(intro,))

In [86]:
file_dataframe = file_dataframe.copy()
file_dataframe['party_type'] = party_type
file_dataframe['word_count'] = word_count
file_dataframe['number_of_questions'] = number_of_questions
file_dataframe['economy'] = economy_true_or_false
#file_dataframe['intro_ending'] = intro_ending_true_or_false
file_dataframe['health_care'] = health_care_true_or_false
file_dataframe['covid_true_or_false'] = covid_true_or_false
file_dataframe['supreme_court_true_or_false'] = supreme_court_true_or_false

file_dataframe

,speaker,minute,text,party_type,word_count,number_of_questions,economy,health_care,covid_true_or_false,supreme_court_true_or_false
0,Chris Wallace,01:20,Good evening from the Health Education Campus ...,Host,124,0,False,False,False,False
1,Chris Wallace,02:10,This debate is being conducted under health an...,Host,102,0,False,False,False,False
2,Vice President Joe Biden,02:49,"How you doing, man?",Democratic,4,1,False,False,False,False
3,President Donald J. Trump,02:51,How are you doing?,Citizen,4,1,False,False,False,False
4,Vice President Joe Biden,02:51,I’m well.,Democratic,2,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
784,Chris Wallace,01:10:43,"Gentlemen, just say that’s the end of it [cros...",Host,17,0,False,False,False,False
785,President Donald J. Trump,01:10:47,I want to see an honest ballot count.,Citizen,8,0,False,False,False,False
786,Chris Wallace,01:10:48,We’re going to leave it there-,Host,6,0,False,False,False,False
787,President Donald J. Trump,01:10:49,And I think he does too-,Citizen,6,0,False,False,False,False
